In [5]:
import json, requests, sys, os, time, re, screed

In [6]:
project_ids = ["mgp19495", "mgp10307", "mgp9220"]


In [7]:
metadata_file = open("tsv/metadata.tsv", "w")
metadata_file.write("\t".join(["sample", "latitude", "longitude", "location", "country", "collection_date", "collection_time", "material", "feature", "metagenome_id"]))
metadata_file.write("\n")
metadata_file.close()

In [8]:
def get_metadata(project_id, export_file):
    metadata_url_prefix = "https://api.mg-rast.org/4/metadata/export/"
    url = f"{metadata_url_prefix}" + project_id
    content = json.loads(requests.get(url).content.decode('iso8859-1'))
    for i in range(len(content["samples"])):
        #print (content["samples"][i]["libraries"][0]["data"]["metagenome_id"]["value"])
        name = content["samples"][i]["id"]
        latitude = content["samples"][i]["data"]["latitude"]["value"]
        longitude = content["samples"][i]["data"]["longitude"]["value"]
        location = content["samples"][i]["data"]["location"]["value"]
        country = content["samples"][i]["data"]["country"]["value"]
        collection_date = content["samples"][i]["data"]["collection_date"]["value"]
        collection_time = content["samples"][i]["data"]["collection_time"]["value"]
        material = content["samples"][i]["data"]["material"]["value"]
        feature = content["samples"][i]["data"]["feature"]["value"]
        metagenome_id = content["samples"][i]["libraries"][0]["data"]["metagenome_id"]["value"]

        metadata_file = open(export_file, "a")
        metadata_file.write("\t".join([f"{name}", f"{latitude}", f"{longitude}", f"{location}", f"{country}", f"{collection_date}", f"{collection_time}", f"{material}", f"{feature}", f"{metagenome_id}"]))
        metadata_file.write("\n")
        metadata_file.close()

In [9]:
for p in project_ids:
    get_metadata(p, "tsv/metadata.tsv")

In [6]:
def get_sequence(sample_id, sample=100, output_folder = "fasta"):
    sequence_url = f"https://api.mg-rast.org/download/{sample_id}?file=299.1"

    content = requests.get(sequence_url).content.decode('iso8859-1')

    fasta_file = open(f"{output_folder}/{sample_id}_original.fasta", "w")
    fasta_file.write(content)
    fasta_file.close()
    
    content = ""
    n = 0
    with screed.open(f"{output_folder}/{sample_id}_original.fasta") as seqfile:
        for read in seqfile:
            if n < 100:
                content += f">{read.name}\n{read.sequence}\n"
                n += 1
            else:
                break
            
    final_fasta_file = open(f"{output_folder}/{sample_id}.fasta", "w")
    final_fasta_file.write(content)
    final_fasta_file.close()

In [7]:
metadata_file = open("tsv/metadata.tsv", "r")

is_header = False

for line in metadata_file:
    if is_header == False:
        is_header = True
    else:
        line = line.strip().split("\t")
        sample_id = line[-1]
        get_sequence(sample_id)

In [10]:

metadata_file = open("tsv/test_metadata.tsv", "w")
metadata_file.write("\t".join(["sample", "latitude", "longitude", "location", "country", "collection_date", "collection_time", "material", "feature", "metagenome_id"]))
metadata_file.write("\n")
metadata_file.close()
get_metadata("mgp103087", "tsv/test_metadata.tsv")